In [23]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, Flatten, Softmax, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
import random

In [25]:
df = open('/kaggle/input/go-competition/29_Training Dataset/Training Dataset/play_style_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]
game_styles = [int(i.split(',',2)[-2]) for i in df]

In [26]:
import os

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [27]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

In [28]:
# 遍历所有游戏数据
for idx, game in enumerate(games):
    num  = 16 
    moves_list = game.split(',')
    # 计算当前游戏的长度
    current_length = len(moves_list)
    # 计算需要添加的空移动数量
    if current_length < num:
        padding_length = (num - (current_length % num)) %num
        # 获取第一个动作作为padding_value
        padding_value = moves_list[0]
        # 使用列表切片添加空移动
        moves_list = [padding_value] * padding_length + moves_list
        # 更新games中的数据
        games[idx] = ','.join(moves_list)

In [30]:
def prepare_input_Zero(moves):
    # 初始化19x19x17的数组
    x = np.zeros((19, 19, 17))
    x_state = np.zeros((19, 19, 2))
    # 历史棋盘状态的索引
    
    history_index = 0
    num = len(moves)
    last_eight_indices = num - 8
    eight_index = moves[last_eight_indices]
    eight_color= eight_index[0]
    for move_num, move in enumerate(moves):
        color = move[0]  
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        # 根据颜色设置对应的历史棋盘层
        if color == 'B':  # 黑子
            x_state[row, column, 0] = 1
        else:  # 白子
            x_state[row, column, 1] = 1
        # 在最后八个移动中，更新历史状态
        if last_eight_indices <= move_num:
            if eight_color == 'B':  # 黑子
                x[:, :, history_index * 2] = x_state[:, :, 0]
                x[:, :, history_index * 2 + 1] = x_state[:, :, 1]
            else:  # 白子
                x[:, :, history_index * 2] = x_state[:, :, 1]
                x[:, :, history_index * 2 + 1] = x_state[:, :, 0]
            history_index += 1
    # 设置当前玩家层
    if moves:
        if eight_color == 'B':  
            x[:, :, -1] = 0  # 白方
        else:
            x[:, :, -1] = 1  # 黑方
    return x

In [31]:
# Check how many samples can be obtained
n_games = 0
for game in games:
    n_games += 1
print(f"Total Games: {n_games}")

Total Games: 26615


In [32]:
x = []
for game in games:
    moves_list = game.split(',')
    x.append(prepare_input_Zero(moves_list))
x = np.array(x)
y = np.array(game_styles)-1

In [33]:
np.bincount(y)

array([8184, 9403, 9028])

In [34]:
x.shape

(26615, 19, 19, 17)

In [35]:
y_hot = tf.one_hot(y, depth=3)
y_hot = y_hot.numpy()

In [37]:
x_train, x_val, y_train, y_val = train_test_split(x, y_hot, test_size=0.01)

In [38]:
def residual_block(x, num_filters):
    # 保存輸入，用於殘差連接
    shortcut = x
    shortcut = Conv2D(num_filters*1, kernel_size=3, padding='same')(shortcut)
    # 第一個卷積層
    x = Conv2D(num_filters, kernel_size=3, padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, kernel_size=3, padding='same')(x)
    x = Activation('relu')(x)

    # 第二個卷積層
    x = Conv2D(num_filters*1, kernel_size=3, padding='same')(x)
    # 殘差連接
    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

In [40]:
def create_model():
    inputs = Input(shape=(19, 19, 17))
    outputs = Conv2D(kernel_size=3, filters=256, padding='same', activation='relu')(inputs)
    for _ in range(8):  # 您可以根據需要調整殘差塊的數量
        outputs = residual_block(outputs, num_filters=256)

    outputs = Flatten()(outputs)
    outputs = Dense(4096, activation='relu')(outputs)
    outputs = Dense(1024, activation='relu')(outputs)
    outputs = Dense(3, activation='softmax', )(outputs)
    model = Model(inputs, outputs)
    opt = Adam(learning_rate=0.00005)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [41]:
model = create_model()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 19, 19, 17)]         0         []                            
                                                                                                  
 conv2d_33 (Conv2D)          (None, 19, 19, 256)          39424     ['input_2[0][0]']             
                                                                                                  
 conv2d_35 (Conv2D)          (None, 19, 19, 256)          590080    ['conv2d_33[0][0]']           
                                                                                                  
 activation_24 (Activation)  (None, 19, 19, 256)          0         ['conv2d_35[0][0]']           
                                                                                            

In [42]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=4,           
    restore_best_weights=True,  
)

In [43]:
from tensorflow.keras.utils import Sequence
import multiprocessing

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
class MyDataGenerator(Sequence):
    def __init__(self, games, batch_size,y):
        self.games = games
        self.y = y
        self.batch_size = batch_size
    def __len__(self):
        return int(np.ceil(len(self.games) / self.batch_size))
    def __getitem__(self, index):
        start = index * self.batch_size
        end = (index + 1) * self.batch_size
        x = self.games[start:end]
        y = self.y[start:end]

        return x, y

In [44]:
batch_size = 64
data_generator_train = MyDataGenerator(x_train, batch_size,y_train)
data_generator_val = MyDataGenerator(x_val, batch_size,y_val)

history = model.fit(
    data_generator_train,
    epochs = 30,
    validation_data=data_generator_val,
    callbacks=[early_stopping],
)

Epoch 1/30
396/396 [==============================] - 113s 250ms/step - loss: 1.0819 - accuracy: 0.3881 - val_loss: 1.0781 - val_accuracy: 0.3914
Epoch 2/30
396/396 [==============================] - 97s 244ms/step - loss: 1.0745 - accuracy: 0.4063 - val_loss: 1.0750 - val_accuracy: 0.4260
Epoch 3/30
396/396 [==============================] - 97s 244ms/step - loss: 1.0724 - accuracy: 0.4168 - val_loss: 1.0715 - val_accuracy: 0.4200
Epoch 4/30
396/396 [==============================] - 97s 244ms/step - loss: 1.0696 - accuracy: 0.4149 - val_loss: 1.0691 - val_accuracy: 0.4350
Epoch 5/30
396/396 [==============================] - 95s 239ms/step - loss: 1.0673 - accuracy: 0.4196 - val_loss: 1.0704 - val_accuracy: 0.4222
Epoch 6/30
396/396 [==============================] - 97s 244ms/step - loss: 1.0592 - accuracy: 0.4309 - val_loss: 1.0443 - val_accuracy: 0.4651
Epoch 7/30
396/396 [==============================] - 97s 244ms/step - loss: 0.9368 - accuracy: 0.5596 - val_loss: 0.8961 - val_a

In [45]:
model.save('./model_playstyle_test.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
